# Main performance experiment for the MLP models using the new ClimateBench metrics
See Sections 4 and 5 of the manuscript

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/USC Random NN')

Mounted at /content/drive


In [ ]:
!pip install eofs --quiet
import numpy as np
import pandas as pd
from utils import *
from utils_nn import count_ffnn_params
results_path = results_path + 'dense/new_metrics_experiment/'

import keras
from keras import Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.utils import plot_model
tf.get_logger().setLevel('ERROR')
from IPython.display import display

tf.keras.utils.set_random_seed(21)

     |████████████████████████████████| 1.0 MB 15.6 MB/s 


In [ ]:
vars_to_predict = ['tas', 'diurnal_temperature_range', 'pr', 'pr90']
# Selects first two years of every decade from 1850 onward as validation
val_idx = np.concatenate((np.arange(0,161,10), np.arange(1,162,10),
                          np.arange(165,326,10), np.arange(166,327,10), 
                          np.arange(330,491,10), np.arange(331,492,10),
                          np.arange(500,571,10), np.arange(501,572,10),
                          np.arange(586,657,10), np.arange(587,658,10),
                          np.arange(672,743,10), np.arange(673,744,10)))

# Create training data
train_files = ['historical', 'hist-GHG', 'hist-aer', 'ssp126', 'ssp370', 'ssp585']
X, solvers = create_predictor_data(train_files, sort_by_time=False)
Y = create_predictdand_data(train_files, sort_by_time=False)
original_shape = Y['tas'].shape  # choice of tas here is arbitrary

X_train_dict = {}
Y_train_dict = {}
X_val_dict = {}
Y_val_dict = {}

for var in vars_to_predict:
  Y_var = Y[var].data.reshape((Y[var].shape[0], -1))
  Y_var = pd.DataFrame(Y_var)
    
  X_val = X.iloc[val_idx]
  X_train = X.drop(index=val_idx)
  Y_val = Y_var.iloc[val_idx]
  Y_train = Y_var.drop(index=val_idx)
    
  X_train_dict[var] = X_train
  X_val_dict[var] = X_val
  Y_train_dict[var] = Y_train
  Y_val_dict[var] = Y_val

# Create test data
X_test = get_test_data('ssp245', solvers)
Y_test = create_predictdand_data(['ssp245'])

X_train_dict['tas'].shape, Y_train_dict['tas'].shape, X_val_dict['tas'].shape, Y_val_dict['tas'].shape

((603, 12), (603, 13824), (150, 12), (150, 13824))

In [ ]:
# THESE ARE SOLELY FOR USE IN RESHAPING THE NN OUTPUT LATER
X_test_np = xr.open_mfdataset([data_path + 'inputs_historical.nc',
                            data_path + 'inputs_ssp245.nc']).compute()
X_test_np = X_test_np.sel(time=range(2015,2101))

In [ ]:
param_lims = {1000000: '1M', 10000000: '10M'}
layer_range = (2,11)
num_models = 50
raw_rmse_data_spatial = []
raw_rmse_data_global = []
raw_rmse_data_total = []

In [ ]:
for param_lim in param_lims.keys():
    
  # param_lim +/- 10%
  param_range = (int(param_lim-0.1*param_lim), int(param_lim+0.1*param_lim))
  rmse_data_spatial = []
  rmse_data_global = []
  rmse_data_total = []

  for num_layers in range(*layer_range):

    for i in range(num_models):
            
      param_count = float('inf')

      while param_count not in range(*param_range):
          # much of the possible range of layer sizes will be too large,
          # so we sample from a smaller space to speed up search
          if param_lim == 1000000:
              units_list = [np.random.randint(12,200)]*num_layers
          elif param_lim == 10000000:
              units_list = [np.random.randint(200,1000)]*num_layers
          units_list = np.insert(units_list,0,12)
          units_list = np.append(units_list,96*144)
          param_count = count_ffnn_params(units_list)

      keras.backend.clear_session()
      untrained_model = None
            
      untrained_model = Sequential()
      untrained_model.add(Input(shape=(12,)))
      for units in units_list[1:-1]:
        untrained_model.add(Dense(units, activation='relu'))
      untrained_model.add(Dense(13824))

      #untrained_model.summary()
      #display(plot_model(untrained_model, show_shapes=True))

      np.save(results_path+f'{param_lims[param_lim]}/models/{num_layers}_layer_model_{i}', untrained_model.get_config())
      plot_model(untrained_model, results_path+f'{param_lims[param_lim]}/images/{num_layers}_layer_model_{i}.png', show_shapes=True)
            
      for var in vars_to_predict:
                
        # Get train/val data
        X_train = X_train_dict[var]
        Y_train = Y_train_dict[var]
        X_val = X_val_dict[var]
        Y_val = Y_val_dict[var]

        model = None
        model = untrained_model
        model.compile(optimizer="adam", loss="mse", metrics=["mse"])
        hist = model.fit(X_train,
                         Y_train,
                         batch_size=16,
                         epochs=100,
                         verbose=0,
                         validation_data=(X_val,Y_val),
                         callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
                        )
        #plot_loss(hist)
        
        # Make predictions using trained model
        m_pred = model.predict(X_test, verbose=0)
        m_pred = m_pred.reshape((-1,) + original_shape[1:])
        m_pred = xr.DataArray(m_pred, dims=['time','lat','lon'], coords= [X_test_np.time.data, X_test_np.latitude.data, X_test_np.longitude.data ])
        m_pred = m_pred.transpose('lat','lon','time').sel(time=slice(2015,2101)).to_dataset(name=var)

        # Save prediction as .nc
        if var == 'diurnal_temperature_range':
          m_pred.to_netcdf(results_path+f'{param_lims[param_lim]}/predictions/dtr/{num_layers}_layer_model_{i}.nc', 'w')
        else:
          m_pred.to_netcdf(results_path+f'{param_lims[param_lim]}/predictions/{var}/{num_layers}_layer_model_{i}.nc', 'w')

        # Calculate RMSE
        var_truth = Y_test[var]
        m_var_pred = m_pred.transpose('time','lat','lon')[var]
        rmse_spatial = get_rmse_spatial(var_truth[65:], m_var_pred[65:])
        raw_rmse_data_spatial.append(rmse_spatial)
        rmse_global = get_rmse_global(var_truth[65:], m_var_pred[65:])
        raw_rmse_data_global.append(rmse_global)
        rmse_total = rmse_spatial + 5*rmse_global
        raw_rmse_data_total.append(rmse_total)
                
    n_layer_rmse_data_spatial = raw_rmse_data_spatial[-(num_models*len(vars_to_predict)):]
    n_layer_rmse_data_spatial = np.reshape(n_layer_rmse_data_spatial, (len(vars_to_predict), num_models), order='F')

    n_layer_rmse_data_global = raw_rmse_data_global[-(num_models*len(vars_to_predict)):]
    n_layer_rmse_data_global = np.reshape(n_layer_rmse_data_global, (len(vars_to_predict), num_models), order='F')

    n_layer_rmse_data_total = raw_rmse_data_total[-(num_models*len(vars_to_predict)):]
    n_layer_rmse_data_total = np.reshape(n_layer_rmse_data_total, (len(vars_to_predict), num_models), order='F')


    print(f'{num_layers} Layer {param_lims[param_lim]} Mean RMSE:')
    for var in vars_to_predict:
        if var == 'diurnal_temperature_range':
            print(f'\tdtr:')
            print(f'\t\tSpatial: {round(np.mean(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(np.mean(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(np.mean(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')
        else:
            print(f'\t{var}:')
            print(f'\t\tSpatial: {round(np.mean(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(np.mean(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(np.mean(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')

    print(f'{num_layers} Layer {param_lims[param_lim]} Min RMSE:')
    for var in vars_to_predict:
        if var == 'diurnal_temperature_range':
            print(f'\tdtr:')
            print(f'\t\tSpatial: {round(min(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(min(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(min(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')
        else:
            print(f'\t{var}:')
            print(f'\t\tSpatial: {round(min(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(min(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(min(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')
            
    rmse_data_spatial.append(n_layer_rmse_data_spatial)
    rmse_data_global.append(n_layer_rmse_data_global)
    rmse_data_total.append(n_layer_rmse_data_total)
        
  np.save(results_path+f'{param_lims[param_lim]}/rmse_data_spatial', rmse_data_spatial)
  np.save(results_path+f'{param_lims[param_lim]}/rmse_data_global', rmse_data_global)
  np.save(results_path+f'{param_lims[param_lim]}/rmse_data_total', rmse_data_total)

2 Layer 1M Mean RMSE:
	tas:
		Spatial: 0.5716
		Global: 0.5006
		Total: 3.0747
	dtr:
		Spatial: 14.5194
		Global: 1.6779
		Total: 22.9088
	pr:
		Spatial: 4.0845
		Global: 0.6217
		Total: 7.1931
	pr90:
		Spatial: 4.8732
		Global: 0.773
		Total: 8.738
2 Layer 1M Min RMSE:
	tas:
		Spatial: 0.433
		Global: 0.3797
		Total: 2.3314
	dtr:
		Spatial: 11.6127
		Global: 0.7816
		Total: 15.6222
	pr:
		Spatial: 3.2566
		Global: 0.1877
		Total: 4.6629
	pr90:
		Spatial: 4.2332
		Global: 0.4556
		Total: 6.7411
3 Layer 1M Mean RMSE:
	tas:
		Spatial: 0.6196
		Global: 0.5464
		Total: 3.3515
	dtr:
		Spatial: 15.1395
		Global: 1.9257
		Total: 24.7679
	pr:
		Spatial: 4.3006
		Global: 0.8266
		Total: 8.4334
	pr90:
		Spatial: 5.0392
		Global: 0.8791
		Total: 9.4349
3 Layer 1M Min RMSE:
	tas:
		Spatial: 0.4648
		Global: 0.4067
		Total: 2.4981
	dtr:
		Spatial: 11.2948
		Global: 0.8167
		Total: 18.3642
	pr:
		Spatial: 3.6711
		Global: 0.5347
		Total: 6.4098
	pr90:
		Spatial: 4.4282
		Global: 0.7
		Total: 8.0512
